## 1️⃣ [Week3/심화] Multi-genre natural language inference(MNLI)

MNLI는 두 문장이 주어졌을 때 논리적으로 연결이 되어 있는지, 서로 모순되는지, 아니면 아예 무관한지 분류하는 문제입니다.

* data : https://www.kaggle.com/datasets/thedevastator/unlocking-language-understanding-with-the-multin

### 1. 데이터 불러오기

In [1]:
import random
import pandas as pd

In [5]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./validation_matched.csv')

In [6]:
print("** train **")
print(f"train len : {len(train_df)}")
print(train_df.columns)
print("** test **")
print(f"test len : {len(test_df)}")

** train **
train len : 392702
Index(['promptID', 'pairID', 'premise', 'premise_binary_parse',
       'premise_parse', 'hypothesis', 'hypothesis_binary_parse',
       'hypothesis_parse', 'genre', 'label'],
      dtype='object')
** test **
test len : 9815


#### 📋 MNLI 데이터셋 칼럼 설명
|칼럼 이름|설명|
|--|--|
|promptID	|원본 문서나 문단 단위의 ID. |
|pairID|	각 문장 쌍의 고유 식별자.|
|premise	|전제 문장.|
|premise_binary_parse	|전제 문장의 binary constituency 구문 트리 (문법 구조 파싱 결과, 괄호로 표현됨).|
|premise_parse	|전제 문장의 full constituency 구문 트리. |
|hypothesis	|가설 문장. premise를 기반으로 이 문장이 참인지 아닌지를 판단|
|hypothesis_binary_parse|	가설 문장의 binary 구문 트리.|
|hypothesis_parse	|가설 문장의 full constituency 구문 트리.|
|genre	|문장 쌍이 나온 도메인/장르 (예: fiction, slate, telephone, travel, government 등). 모델이 도메인 일반화 능력을 가질 수 있게 도와주는 요소.|
|label|	정답 레이블. entailment, contradiction, neutral, 또는 - (레이블 없음: 예측 대상일 때 사용).|

결굴 premise와 hypothesis의 관계를 pred 해야하는 것이고 정답은 label에 있으므로 이 3개 column을 사용하면 된다.

##### 데이터 확인 및 전처리

* 결측치 확인
* train 데이터 분포 확인 (불균형한지)
* label의 의미 확인

In [7]:
train_df = train_df[['premise','hypothesis','label']]
test_df = test_df[['premise','hypothesis','label']]
train_df.head()

,premise,hypothesis,label
0,Conceptually cream skimming has two basic dime...,Product and geography are what make cream skim...,1
1,you know during the season and i guess at at y...,You lose the things to the following level if ...,0
2,One of our number will carry out your instruct...,A member of my team will execute your orders w...,0
3,How do you know? All this is their information...,This information belongs to them.,0
4,yeah i tell you what though if you go price so...,The tennis shoes have a range of prices.,1


In [9]:
print("** train **")
print(train_df.isnull().sum())
print("** test **")
print(test_df.isnull().sum())

** train **
premise        0
hypothesis    40
label          0
dtype: int64
** test **
premise       0
hypothesis    0
label         0
dtype: int64


In [10]:
## 결측치 제거
before_len = len(train_df)
train_df = train_df.dropna()
after_len = len(train_df)

print(f"{before_len - after_len} rows dropped.")

40 rows dropped.


* 결측치를 제거하고 label이 값이 3개인지 확인한다.

In [11]:
train_df['label'].unique()

array([1, 0, 2])

* train 데이터가 불균형한지도 확인해준다.

In [13]:
train_df['label'].value_counts()

,count
label,
2,130889
1,130887
0,130886


In [14]:
for label in train_df['label'].unique():
    examples = train_df[train_df['label'] == label].sample(3)
    print(f"\nLabel {label} 예시:")
    for i, row in examples.iterrows():
        print(f"- Premise: {row['premise']}")
        print(f"  Hypothesis: {row['hypothesis']}")


Label 1 예시:
- Premise: Also, during the first three months of 1997, she had met twice with the organization's chief executive officer, at his request, to discuss the security implications of new applications.
  Hypothesis: She first met the chief executive officer in early February.
- Premise: A small one, sir.
  Hypothesis: I can only handle a small one.
- Premise: But since you have proven exponentially easier to track and catch than White, I severely doubt you're using the same underground network.
  Hypothesis: We found White last week.

Label 0 예시:
- Premise: Adequately warned, why do people persist in sucking cancer-causing tars into their lungs?
  Hypothesis: Despite adequate warnings, people continue to smoke cigarettes.
- Premise: Acknowledging reality isn't unethical, but ignoring it can be.
  Hypothesis: Ignoring reality can be unethical.
- Premise: was going to run me five hundred bucks
  Hypothesis: Was going to cost me 500 dollars.

Label 2 예시:
- Premise: The qa'a led to

어떤 라벨이 어떤 뜻인지 헷갈렸는데 예시 찍어보고 다시 확인

* label 0 - Entailment
  * 의미가 거의 같거나, 하나가 다른 하나로부터 도출됨
* label 1 - Neutral
* Label 2 - Contradiction
  * 전제와 가설이 명백히 상반됨

### 2. dataset 준비

(여러 모델을 테스트 하기 위해 재사용성 있는 코드)
* tokenizer 불러오기


In [15]:
from transformers import AutoTokenizer

def get_tokenizer(model_name):
    return AutoTokenizer.from_pretrained(model_name)

* dataset 만들기
 * collate_fn 안에서 여러 처리를 해주고 이 안에선 pair로만 만들어준다.

In [17]:
from torch.utils.data import Dataset

class Dataset_Maker(Dataset):
    def __init__(self, df):
        self.pairs = list(zip(df['premise'], df['hypothesis']))
        self.labels = df['label'].tolist()

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'text_pair': self.pairs[idx],
            'label': self.labels[idx]
        }

* collate_fn
  * premises와 hypotheses를 한 문장 쌍으로 만듬
  * 패딩 처리
  * truncation 처리
  * masking은 attention_mask 사용

In [18]:
import torch

def collate_fn(batch, tokenizer, max_length=256):
    premises, hypotheses = zip(*[item['text_pair'] for item in batch])
    labels = [item['label'] for item in batch]

    encodings = tokenizer(
        list(premises),
        list(hypotheses),
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors='pt'
    )
    encodings['labels'] = torch.tensor(labels, dtype=torch.long)
    return encodings

tokenizer의 max_length를 확인해서 collate_fn에 넘겨주어 모델을 바꾸더라도 문제가 없도록 처리해준다.

In [19]:
model_name = "distilbert-base-uncased"

In [21]:
from torch.utils.data import DataLoader

tokenizer = get_tokenizer(model_name)

def get_dataloader(tokenizer, train_df, test_df, batch_size):
    max_length = tokenizer.model_max_length

    train_ds = Dataset_Maker(train_df)
    train_loader = DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=lambda batch: collate_fn(batch, tokenizer, max_length)
    )

    test_ds = Dataset_Maker(test_df)
    test_loader = DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=lambda batch: collate_fn(batch, tokenizer, max_length)
    )
    return train_loader, test_loader

train_loader, test_loader = get_dataloader(tokenizer, train_df, test_df, 32)

### 3. Model 구성
* pretrained model 불러오기
* MNLIClassifier 구조잡기

In [22]:
import torch.nn as nn
from transformers import AutoModel

* token_type_ids
  * BERT 에서 NLI 문제에서 pretrained 된 것 문장 구분을 위한 type_ids
  * 일단 distilBert로 진행하지만 BERT 로 실험을 해볼수도 있어서 추가해줌

In [23]:
class MNLIClassifier(nn.Module):
    def __init__(self, model_name):
        super().__init__()
        self.pretrained_model = AutoModel.from_pretrained(model_name)
        hidden_size = self.pretrained_model.config.hidden_size

        self.classifier = nn.Linear(hidden_size, 3)

        print(f"** load pretrained model {model_name} **")
        print(self.pretrained_model)

    def forward(self, input_ids, attention_mask, token_type_ids=None):
        outputs = self.pretrained_model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids if token_type_ids is not None else None
        )
        cls_output = outputs.last_hidden_state[:, 0, :]  # CLS 위치
        pred_output = self.classifier(cls_output)
        return pred_output
